In [1]:
!pip install transformers datasets accelerate rouge_score torch nltk bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-1

In [3]:
import torch
from torch import nn
from transformers import (AutoTokenizer , AutoModel ,AutoModelForSeq2SeqLM ,
                          DataCollatorForSeq2Seq , Seq2SeqTrainingArguments
                          , Seq2SeqTrainer  , BitsAndBytesConfig  )
from peft import LoraConfig   , get_peft_model , TaskType
from datasets import load_dataset , load_metric
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#load cleaned datasets
print("Downloading data ...")
xlsum_arabic = load_dataset("Mohamed-Sami/cleaned_xlsum_arabic")
print("data is downloaded !")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32877 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4547 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4559 [00:00<?, ? examples/s]

data is downloaded !


In [5]:
xlsum_arabic

DatasetDict({
    train: Dataset({
        features: ['summary', 'text'],
        num_rows: 32877
    })
    test: Dataset({
        features: ['summary', 'text'],
        num_rows: 4547
    })
    validation: Dataset({
        features: ['summary', 'text'],
        num_rows: 4559
    })
})

In [6]:
torch.cuda.is_available()

False

In [7]:
BitsAndBytesConfig?

In [8]:
model_checkpoint = "google/mt5-small"
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True , bnb_4bit_use_double_quant=True ,
    bnb_4bit_quant_type='nf4' , bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint , device_map={"":0} ,
                                              quantization_config = quantization_config)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
#prepera data
max_text_length = 1024
max_summary_length = 40
def tokenize_data(examples):

  input_encoding = tokenizer(examples['text'] , max_length = max_text_length , truncation =True , padding =True)

  with tokenizer.as_target_tokenizer():

    summary_encoding = tokenizer(examples['summary'] , max_length = max_summary_length , truncation =True , padding= True)

  return {'input_ids' : input_encoding['input_ids'] ,
          'attention_mask' : input_encoding['attention_mask'] ,
          'labels' : summary_encoding['input_ids'] }
          #'decoder_input_ids':  summary_encoding['input_ids']}


In [ ]:
xlsum_ar_datasets = xlsum_arabic.map(tokenize_data , batched = True , remove_columns=['text' , 'summary'])

Map:   0%|          | 0/32877 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4547 [00:00<?, ? examples/s]

Map:   0%|          | 0/4559 [00:00<?, ? examples/s]

In [ ]:
xlsum_ar_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32877
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4547
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4559
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer , model = model , return_tensors= "pt")

In [ ]:
trainig_args = Seq2SeqTrainingArguments(
    output_dir = 'mt5-xlsum-arabic' , num_train_epochs = 1 , warmup_ratio = 0.01 ,
    per_device_train_batch_size = 1 ,
    weight_decay = 0.01 , logging_steps = 10
    , learning_rate = 5e-5 ,  gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
rouge_score = load_metric("rouge")

<ipython-input-13-3f83cc71b72a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_score = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = [ pred for pred in decoded_preds]
    decoded_labels = [ label for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    trainig_args,
    train_dataset=xlsum_ar_datasets["train"],
    #eval_dataset=xlsum_ar_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

In [ ]:
import gc
gc.collect()

93

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 222.00 MiB. GPU 